# Managing population growth

In [1]:
import sys
import numpy as np
import pandas as pd
import altair as alt
# mapping to the modules that make the app
sys.path.insert(0, "../..")
sys.path.insert(0, "../../app")


import yaml, ssl, asyncio, pickle, os, ast


ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())

import nest_asyncio
# this is required for running in a Jupyter Notebook. 
nest_asyncio.apply()

In [2]:
# mapping to the modules that make the app

sys.path.insert(0, "../..")
sys.path.insert(0, "../../app")
sys.path.insert(0, "../../functions/popgrowth/")

import cmdb_graph

c = cmdb_graph.CosmosdbClient()

In [3]:
params = yaml.safe_load(open(os.path.join(os.getenv("abspath"),"functions/popgrowth/settings.yml")))
syllables = pickle.load(open(os.path.join(os.getenv("abspath"),"app/creators/specs/syllables.p"), "rb"))
username = 'notebook'
params

{'pop_health_requirement': 0.6,
 'pop_consumes': 2,
 'starve_damage': 0.05,
 'changing_values': ['conformity',
  'literacy',
  'aggression',
  'constitution',
  'wealth',
  'factionLoyalty']}

Time is updated by the `time` function. It is always running.

In [4]:
c.run_query("g.V().hasLabel('time').valueMap()")
params['time'] = c.clean_nodes(c.res)[0]
params['time']

{'currentTime': 28290,
 'updatedFrom': 'azfunction',
 'updatedAt': '2023-05-19 09:10:00.380788',
 'objid': '7263327370874',
 'name': 'time',
 'originTime': 0,
 'username': 'notebook',
 'objtype': 'time',
 'id': '7263327370874'}

For this notebook, setting the `pop_health_requirement` to an arbitrary value just to examine.

In [5]:
params['pop_health_requirement'] = 0.4

## Population Resource Consumption

In [6]:
from tools import consumption

We're going to run a resourse query test to ensure that consumption is happening. To test, update the `objid` with a planet that has consuming pops. 

In [7]:
validate_resources_updated_query = f"""
g.V().has('objid','2972908340606').out('has').valueMap()
"""
print(validate_resources_updated_query)
c.run_query(validate_resources_updated_query)
pre_consumption = pd.DataFrame.from_records(c.clean_nodes(c.res))
pre_consumption


g.V().has('objid','2972908340606').out('has').valueMap()



,volume,name,objid,max_volume,description,replenish_rate,username,objtype,id
0,891,Organic,8679787925290,1013,bilogical material that can be consumed by pops,10.0,BillmanLocal2,resource,8679787925290
1,96,Common Minerals,7928654046397,96,Iron and other common material used in constru...,NaN,BillmanLocal2,resource,7928654046397
2,54,Rare Minerals,3119228965862,54,"lithium, silver and other rare minerals used i...",NaN,BillmanLocal2,resource,3119228965862
3,9626,Water,1324452532458,9626,"H2O ready to be consumed, either frozen or in ...",NaN,BillmanLocal2,resource,1324452532458


In [8]:

all_pops_query = f"""
g.V().has('label','pop').as('pop')
    .local(
        union(
            out('inhabits').as('location'),
            out('isOf').as('species')
            )
            .fold()).as('location','species')
        .path()
        .by(unfold().valueMap().fold())
"""

In [9]:
c.run_query(all_pops_query)
data = c.reduce_res(c.res)
pd.DataFrame(data)

,pop,location,species
0,"{'name': 'Bames Todin', 'objid': '437668149746...","{'name': 'Townnak', 'class': 'terrestrial', 'o...","{'name': 'Baydybor', 'objid': '5385225260536',..."
1,"{'name': 'Zokekegua Long', 'objid': '664458242...","{'name': 'Townnak', 'class': 'terrestrial', 'o...","{'name': 'Baydybor', 'objid': '5385225260536',..."
2,"{'name': 'Bames Caslay', 'objid': '39838089911...","{'name': 'Townnak', 'class': 'terrestrial', 'o...","{'name': 'Baydybor', 'objid': '5385225260536',..."
3,"{'name': 'Bames Mesganj', 'objid': '8182730130...","{'name': 'Townnak', 'class': 'terrestrial', 'o...","{'name': 'Baydybor', 'objid': '5385225260536',..."
4,"{'name': 'Bames Fordciubeitsu', 'objid': '4068...","{'name': 'Townnak', 'class': 'terrestrial', 'o...","{'name': 'Baydybor', 'objid': '5385225260536',..."
5,"{'name': 'Zokekegua Bang', 'objid': '635437469...","{'name': 'Townnak', 'class': 'terrestrial', 'o...","{'name': 'Baydybor', 'objid': '5385225260536',..."
6,"{'name': 'Zokekegua Piakoy', 'objid': '0657651...","{'name': 'Townnak', 'class': 'terrestrial', 'o...","{'name': 'Baydybor', 'objid': '5385225260536',..."
7,"{'name': 'Rayahhun Frantiae', 'objid': '760304...","{'name': 'Jivar', 'class': 'terrestrial', 'obj...","{'name': 'East', 'objid': '0161899935756', 'co..."
8,"{'name': 'Kalgagerkal Bedeldepark', 'objid': '...","{'name': 'Jivar', 'class': 'terrestrial', 'obj...","{'name': 'East', 'objid': '0161899935756', 'co..."
9,"{'name': 'Rayahhun Dorf', 'objid': '3760594787...","{'name': 'Jivar', 'class': 'terrestrial', 'obj...","{'name': 'East', 'objid': '0161899935756', 'co..."


In [10]:
pops_df = pd.DataFrame([d['pop'] for d in data])
pops_df

,name,objid,conformity,literacy,aggression,constitution,health,isIn,industry,wealth,factionLoyalty,isIdle,username,objtype,id
0,Bames Todin,4376681497460,0.408,0.469,0.474,0.380,0.70,3820091516344,0.4270,0.4480,0.373,true,Billmanserver,pop,4376681497460
1,Zokekegua Long,6644582427941,0.647,0.361,0.526,0.532,0.70,4353938110450,0.5290,0.4450,0.651,true,Billmanserver,pop,6644582427941
2,Bames Caslay,3983808991145,0.394,0.496,0.448,0.602,0.70,3820091516344,0.5250,0.5105,0.361,true,Billmanserver,pop,3983808991145
3,Bames Mesganj,8182730130672,0.456,0.555,0.613,0.496,0.70,3820091516344,0.5545,0.5548,0.541,true,Billmanserver,pop,8182730130672
4,Bames Fordciubeitsu,4068023816136,0.436,0.575,0.420,0.529,0.70,3820091516344,0.4745,0.5248,0.399,true,Billmanserver,pop,4068023816136
5,Zokekegua Bang,6354374692878,0.467,0.288,0.439,0.532,0.70,4353938110450,0.4855,0.3868,0.571,true,Billmanserver,pop,6354374692878
6,Zokekegua Piakoy,0657651821206,0.570,0.461,0.428,0.548,0.70,4353938110450,0.4880,0.4745,0.575,true,Billmanserver,pop,0657651821206
7,Rayahhun Frantiae,7603049807934,0.634,0.356,0.504,0.493,0.70,5821792704095,0.4985,0.4272,0.613,true,william.jeffrey.harding@gmail.com,pop,7603049807934
8,Kalgagerkal Bedeldepark,0653635598459,0.202,0.462,0.479,0.600,0.70,6730711275117,0.5395,0.5008,0.056,true,william.jeffrey.harding@gmail.com,pop,0653635598459
9,Rayahhun Dorf,3760594787864,0.538,0.310,0.549,0.360,0.70,5821792704095,0.4545,0.3822,0.746,true,william.jeffrey.harding@gmail.com,pop,3760594787864


In [11]:
species_df = pd.DataFrame([d['species'] for d in data])
species_df

,name,objid,consumes,effuses,viral_resilience,habitat_resilience,username,objtype,id
0,Baydybor,5385225260536,[Organic],"[Organic waste, Plastics]",0.7,0.2,Billmanserver,species,5385225260536
1,Baydybor,5385225260536,[Organic],"[Organic waste, Plastics]",0.7,0.2,Billmanserver,species,5385225260536
2,Baydybor,5385225260536,[Organic],"[Organic waste, Plastics]",0.7,0.2,Billmanserver,species,5385225260536
3,Baydybor,5385225260536,[Organic],"[Organic waste, Plastics]",0.7,0.2,Billmanserver,species,5385225260536
4,Baydybor,5385225260536,[Organic],"[Organic waste, Plastics]",0.7,0.2,Billmanserver,species,5385225260536
5,Baydybor,5385225260536,[Organic],"[Organic waste, Plastics]",0.7,0.2,Billmanserver,species,5385225260536
6,Baydybor,5385225260536,[Organic],"[Organic waste, Plastics]",0.7,0.2,Billmanserver,species,5385225260536
7,East,0161899935756,[Organic],"[Organic waste, Plastics]",0.7,0.2,william.jeffrey.harding@gmail.com,species,0161899935756
8,East,0161899935756,[Organic],"[Organic waste, Plastics]",0.7,0.2,william.jeffrey.harding@gmail.com,species,0161899935756
9,East,0161899935756,[Organic],"[Organic waste, Plastics]",0.7,0.2,william.jeffrey.harding@gmail.com,species,0161899935756


In [12]:
locations_df = pd.DataFrame([d['location'] for d in data])
locations_df

,name,class,objid,radius,mass,orbitsDistance,orbitsId,orbitsName,isSupportsLife,isPopulated,isHomeworld,username,objtype,id
0,Townnak,terrestrial,2793144513261,0.701,0.884,0.490,6092232618401,Woodling,true,true,true,Billmanserver,planet,2793144513261
1,Townnak,terrestrial,2793144513261,0.701,0.884,0.490,6092232618401,Woodling,true,true,true,Billmanserver,planet,2793144513261
2,Townnak,terrestrial,2793144513261,0.701,0.884,0.490,6092232618401,Woodling,true,true,true,Billmanserver,planet,2793144513261
3,Townnak,terrestrial,2793144513261,0.701,0.884,0.490,6092232618401,Woodling,true,true,true,Billmanserver,planet,2793144513261
4,Townnak,terrestrial,2793144513261,0.701,0.884,0.490,6092232618401,Woodling,true,true,true,Billmanserver,planet,2793144513261
5,Townnak,terrestrial,2793144513261,0.701,0.884,0.490,6092232618401,Woodling,true,true,true,Billmanserver,planet,2793144513261
6,Townnak,terrestrial,2793144513261,0.701,0.884,0.490,6092232618401,Woodling,true,true,true,Billmanserver,planet,2793144513261
7,Jivar,terrestrial,3493185958037,0.882,0.136,0.899,9585105280215,Zache,true,true,true,william.jeffrey.harding@gmail.com,planet,3493185958037
8,Jivar,terrestrial,3493185958037,0.882,0.136,0.899,9585105280215,Zache,true,true,true,william.jeffrey.harding@gmail.com,planet,3493185958037
9,Jivar,terrestrial,3493185958037,0.882,0.136,0.899,9585105280215,Zache,true,true,true,william.jeffrey.harding@gmail.com,planet,3493185958037


First getting the list of consumption for each pop.


In [13]:
pops_df,species_df,locations_df = consumption.all_pops_consumption(c)


We expand that dataset to break open the list of people in each pop. 

In [14]:
params

{'pop_health_requirement': 0.4,
 'pop_consumes': 2,
 'starve_damage': 0.05,
 'changing_values': ['conformity',
  'literacy',
  'aggression',
  'constitution',
  'wealth',
  'factionLoyalty'],
 'time': {'currentTime': 28290,
  'updatedFrom': 'azfunction',
  'updatedAt': '2023-05-19 09:10:00.380788',
  'objid': '7263327370874',
  'name': 'time',
  'originTime': 0,
  'username': 'notebook',
  'objtype': 'time',
  'id': '7263327370874'}}

In [15]:
consumption_df = consumption.get_consumption_df(locations_df,species_df,params)
consumption_df = consumption.expand_consumption_df(consumption_df)
consumption_df

c:\Users\william.harding\Documents\repos\exoplanets\notebooks\People\../../functions/popgrowth\tools\consumption.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ser.consumes = j
c:\Users\william.harding\Documents\repos\exoplanets\notebooks\People\../../functions/popgrowth\tools\consumption.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ser.consumes = j
c:\Users\william.harding\Documents\repos\exoplanets\notebooks\People\../../functions/popgrowth\tools\consumption.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pa

,location_id,consumes,pop,consumption,multi
0,2793144513261,Organic,17,34,True
1,2972908340606,Organic,16,32,True
2,3493185958037,Organic,13,26,True


Then we get a list of the resources available on that location. 

In [16]:
c.run_query(consumption.make_resource_query(consumption_df))
resources = c.res
resources

[{'labels': [['location'], ['resource']],
  'objects': [{'objid': ['2793144513261'], 'name': ['Townnak']},
   {'volume': [869], 'objid': ['4741040123642'], 'name': ['Organic']}]},
 {'labels': [['location'], ['resource']],
  'objects': [{'objid': ['3493185958037'], 'name': ['Jivar']},
   {'volume': [947], 'objid': ['4370165806739'], 'name': ['Organic']}]},
 {'labels': [['location'], ['resource']],
  'objects': [{'objid': ['2972908340606'], 'name': ['Dorf']},
   {'volume': [891], 'objid': ['8679787925290'], 'name': ['Organic']}]}]

In [17]:
consumption_df = consumption.tally_consumption(c,consumption_df,resources)
consumption_df

,location_id,consumes,pop,consumption,multi,available,remaining
0,2793144513261,Organic,17,34,True,869.0,835.0
1,2972908340606,Organic,16,32,True,891.0,859.0
2,3493185958037,Organic,13,26,True,947.0,921.0


Then we update the resources by taking out the resources that exist. You can check that this has happened in the graph by looking at the location ids. This is a verification query that isn't run in the function application.

In [18]:
consumption_df.apply(lambda x: consumption.make_resource_update_query(c,x),axis=1)

0    None
1    None
2    None
dtype: object

In [19]:
c.run_query(validate_resources_updated_query)
post_consumption = pd.DataFrame.from_records(c.clean_nodes(c.res))

In [20]:
pre_consumption

,volume,name,objid,max_volume,description,replenish_rate,username,objtype,id
0,891,Organic,8679787925290,1013,bilogical material that can be consumed by pops,10.0,BillmanLocal2,resource,8679787925290
1,96,Common Minerals,7928654046397,96,Iron and other common material used in constru...,NaN,BillmanLocal2,resource,7928654046397
2,54,Rare Minerals,3119228965862,54,"lithium, silver and other rare minerals used i...",NaN,BillmanLocal2,resource,3119228965862
3,9626,Water,1324452532458,9626,"H2O ready to be consumed, either frozen or in ...",NaN,BillmanLocal2,resource,1324452532458


In [21]:
post_consumption

,volume,name,objid,max_volume,description,replenish_rate,username,objtype,id
0,859,Organic,8679787925290,1013,bilogical material that can be consumed by pops,10.0,BillmanLocal2,resource,8679787925290
1,96,Common Minerals,7928654046397,96,Iron and other common material used in constru...,NaN,BillmanLocal2,resource,7928654046397
2,54,Rare Minerals,3119228965862,54,"lithium, silver and other rare minerals used i...",NaN,BillmanLocal2,resource,3119228965862
3,9626,Water,1324452532458,9626,"H2O ready to be consumed, either frozen or in ...",NaN,BillmanLocal2,resource,1324452532458


Now for the starving. Populations without enough food will starve (health declines). Populations with < 0 health will die.

In [22]:
consumption_df

,location_id,consumes,pop,consumption,multi,available,remaining
0,2793144513261,Organic,17,34,True,869.0,835.0
1,2972908340606,Organic,16,32,True,891.0,859.0
2,3493185958037,Organic,13,26,True,947.0,921.0


In [23]:
consumption_df[consumption_df['remaining']<=0].apply(lambda x: consumption.lower_health(c,params,x),axis=1)

0 pops will starve in nan


,location_id,consumes,pop,consumption,multi,available,remaining


Looking more closely at `lower_health`
This only happens on locations that have < 0 resources. But here we're going to do it regardless

In [24]:
x = consumption_df.loc[0]
print(x)

dead_pop_nodes = []
dead_pop_ids = []
death_event_edges = []
query =f"""
g.V().has('objid','{x.location_id}').as('location').in('inhabits')
    .haslabel('pop').as('pop')
    .out('isOfSpecies').as('species')
    .path()
        .by(valueMap('objid','name'))
        .by(valueMap('name','objid','health','username'))
        .by(valueMap('name','objid','consumes'))
"""
c.run_query(query)
out = c.res
out

location_id    2793144513261
consumes             Organic
pop                       17
consumption               34
multi                   True
available              869.0
remaining              835.0
Name: 0, dtype: object


[]

In [25]:
print(f"{len(out)} pops will starve in {x.location_id}")

0 pops will starve in 2793144513261


We are only going to starve the pop if the pop consumes that resource. 

health just goes down. 

Let's pretend that the pop will die. IF they do, it will create a `death event`.

In [26]:
# death_event = consumption.death_by_starvation_event(c.clean_node(i['objects'][0]),i['objects'][1],params)
# death_event

We will also link that event to an appropriate `happendAt` so that we can see it when looking at events related to specific things. 

In [27]:
# print(c.create_custom_edge(death_event, c.clean_node(i['objects'][0]), 'happenedAt').strip())

## Population growth
* Populations that die are deleted, so they don't grow.
* However all pops that can grow will, regardless of wheter they are starving or not. 
* Starving will lower thier health, which will eventually make them unable to grow. 


In [28]:
from tools import growth


In [29]:
pops_df,species_df,locations_df,factions_df = growth.get_pop_health(c,params)
print(f"lenth of pops == 0 : {len(pops_df)==0}")
pops_df

lenth of pops == 0 : False


,health,name,objid,conformity,literacy,aggression,constitution,isIn,industry,wealth,factionLoyalty,isIdle,username,objtype,id
0,0.7,Bames Todin,4376681497460,0.408,0.469,0.474,0.380,3820091516344,0.4270,0.4480,0.373,true,Billmanserver,pop,4376681497460
1,0.7,Zokekegua Long,6644582427941,0.647,0.361,0.526,0.532,4353938110450,0.5290,0.4450,0.651,true,Billmanserver,pop,6644582427941
2,0.7,Bames Caslay,3983808991145,0.394,0.496,0.448,0.602,3820091516344,0.5250,0.5105,0.361,true,Billmanserver,pop,3983808991145
3,0.7,Bames Mesganj,8182730130672,0.456,0.555,0.613,0.496,3820091516344,0.5545,0.5548,0.541,true,Billmanserver,pop,8182730130672
4,0.7,Bames Fordciubeitsu,4068023816136,0.436,0.575,0.420,0.529,3820091516344,0.4745,0.5248,0.399,true,Billmanserver,pop,4068023816136
5,0.7,Zokekegua Bang,6354374692878,0.467,0.288,0.439,0.532,4353938110450,0.4855,0.3868,0.571,true,Billmanserver,pop,6354374692878
6,0.7,Zokekegua Piakoy,0657651821206,0.570,0.461,0.428,0.548,4353938110450,0.4880,0.4745,0.575,true,Billmanserver,pop,0657651821206
7,0.7,Rayahhun Frantiae,7603049807934,0.634,0.356,0.504,0.493,5821792704095,0.4985,0.4272,0.613,true,william.jeffrey.harding@gmail.com,pop,7603049807934
8,0.7,Kalgagerkal Bedeldepark,0653635598459,0.202,0.462,0.479,0.600,6730711275117,0.5395,0.5008,0.056,true,william.jeffrey.harding@gmail.com,pop,0653635598459
9,0.7,Rayahhun Dorf,3760594787864,0.538,0.310,0.549,0.360,5821792704095,0.4545,0.3822,0.746,true,william.jeffrey.harding@gmail.com,pop,3760594787864


In [30]:
species_df

,name,objid,consumes,effuses,viral_resilience,habitat_resilience,username,objtype,id
0,Baydybor,5385225260536,[Organic],"[Organic waste, Plastics]",0.7,0.2,Billmanserver,species,5385225260536
1,Baydybor,5385225260536,[Organic],"[Organic waste, Plastics]",0.7,0.2,Billmanserver,species,5385225260536
2,Baydybor,5385225260536,[Organic],"[Organic waste, Plastics]",0.7,0.2,Billmanserver,species,5385225260536
3,Baydybor,5385225260536,[Organic],"[Organic waste, Plastics]",0.7,0.2,Billmanserver,species,5385225260536
4,Baydybor,5385225260536,[Organic],"[Organic waste, Plastics]",0.7,0.2,Billmanserver,species,5385225260536
5,Baydybor,5385225260536,[Organic],"[Organic waste, Plastics]",0.7,0.2,Billmanserver,species,5385225260536
6,Baydybor,5385225260536,[Organic],"[Organic waste, Plastics]",0.7,0.2,Billmanserver,species,5385225260536
7,East,0161899935756,[Organic],"[Organic waste, Plastics]",0.7,0.2,william.jeffrey.harding@gmail.com,species,0161899935756
8,East,0161899935756,[Organic],"[Organic waste, Plastics]",0.7,0.2,william.jeffrey.harding@gmail.com,species,0161899935756
9,East,0161899935756,[Organic],"[Organic waste, Plastics]",0.7,0.2,william.jeffrey.harding@gmail.com,species,0161899935756


In [31]:
locations_df

,name,class,objid,radius,mass,orbitsDistance,orbitsId,orbitsName,isSupportsLife,isPopulated,isHomeworld,username,objtype,id
0,Townnak,terrestrial,2793144513261,0.701,0.884,0.490,6092232618401,Woodling,true,true,true,Billmanserver,planet,2793144513261
1,Townnak,terrestrial,2793144513261,0.701,0.884,0.490,6092232618401,Woodling,true,true,true,Billmanserver,planet,2793144513261
2,Townnak,terrestrial,2793144513261,0.701,0.884,0.490,6092232618401,Woodling,true,true,true,Billmanserver,planet,2793144513261
3,Townnak,terrestrial,2793144513261,0.701,0.884,0.490,6092232618401,Woodling,true,true,true,Billmanserver,planet,2793144513261
4,Townnak,terrestrial,2793144513261,0.701,0.884,0.490,6092232618401,Woodling,true,true,true,Billmanserver,planet,2793144513261
5,Townnak,terrestrial,2793144513261,0.701,0.884,0.490,6092232618401,Woodling,true,true,true,Billmanserver,planet,2793144513261
6,Townnak,terrestrial,2793144513261,0.701,0.884,0.490,6092232618401,Woodling,true,true,true,Billmanserver,planet,2793144513261
7,Jivar,terrestrial,3493185958037,0.882,0.136,0.899,9585105280215,Zache,true,true,true,william.jeffrey.harding@gmail.com,planet,3493185958037
8,Jivar,terrestrial,3493185958037,0.882,0.136,0.899,9585105280215,Zache,true,true,true,william.jeffrey.harding@gmail.com,planet,3493185958037
9,Jivar,terrestrial,3493185958037,0.882,0.136,0.899,9585105280215,Zache,true,true,true,william.jeffrey.harding@gmail.com,planet,3493185958037


In [32]:
factions_df

,name,objid,lat,long,pop_loactions,username,objtype,id,pop_locations
0,Bames,3820091516344,0.105,0.0,"[[0, 0], [-1, -1], [-2, 0], [0, 2], [-1, 1]]",Billmanserver,faction,3820091516344,NaN
1,Zokekegua,4353938110450,-0.105,0.0,"[[0, 0], [2, 0], [-2, 0], [0, -2]]",Billmanserver,faction,4353938110450,NaN
2,Bames,3820091516344,0.105,0.0,"[[0, 0], [-1, -1], [-2, 0], [0, 2], [-1, 1]]",Billmanserver,faction,3820091516344,NaN
3,Bames,3820091516344,0.105,0.0,"[[0, 0], [-1, -1], [-2, 0], [0, 2], [-1, 1]]",Billmanserver,faction,3820091516344,NaN
4,Bames,3820091516344,0.105,0.0,"[[0, 0], [-1, -1], [-2, 0], [0, 2], [-1, 1]]",Billmanserver,faction,3820091516344,NaN
5,Zokekegua,4353938110450,-0.105,0.0,"[[0, 0], [2, 0], [-2, 0], [0, -2]]",Billmanserver,faction,4353938110450,NaN
6,Zokekegua,4353938110450,-0.105,0.0,"[[0, 0], [2, 0], [-2, 0], [0, -2]]",Billmanserver,faction,4353938110450,NaN
7,Rayahhun,5821792704095,-0.205,0.0,"[[0, 0], [-1, -1], [-2, -2], [-2, 2], [0, -2],...",william.jeffrey.harding@gmail.com,faction,5821792704095,NaN
8,Kalgagerkal,6730711275117,0.205,0.0,"[[0, 0], [-2, 0]]",william.jeffrey.harding@gmail.com,faction,6730711275117,NaN
9,Rayahhun,5821792704095,-0.205,0.0,"[[0, 0], [-1, -1], [-2, -2], [-2, 2], [0, -2],...",william.jeffrey.harding@gmail.com,faction,5821792704095,NaN


In [33]:
factions_df['pop_locations']

0                                                   NaN
1                                                   NaN
2                                                   NaN
3                                                   NaN
4                                                   NaN
5                                                   NaN
6                                                   NaN
7                                                   NaN
8                                                   NaN
9                                                   NaN
10                                                  NaN
11                                                  NaN
12                                                  NaN
13                                                  NaN
14    [[0, 0], [-1, 1], [-2, 0], [0, 2], [-1, -1], [...
15    [[0, 0], [-1, 1], [-2, 0], [0, 2], [-1, -1], [...
16    [[0, 0], [-1, 1], [-2, 0], [0, 2], [-1, -1], [...
17    [[0, 0], [-1, 1], [-2, 0], [0, 2], [-1, -1

In [37]:
pops_df['roll'] = pops_df['objid'].apply(lambda x: np.random.random())
pops_df['grow'] = pops_df[['wealth','health']].T.mean() >= pops_df['roll']

reproducing_pops = pops_df[pops_df['grow']].drop(['roll','grow'],axis=1)
pops_df

,health,name,objid,conformity,literacy,aggression,constitution,isIn,industry,wealth,factionLoyalty,isIdle,username,objtype,id,roll,grow
0,0.7,Bames Todin,4376681497460,0.408,0.469,0.474,0.380,3820091516344,0.4270,0.4480,0.373,true,Billmanserver,pop,4376681497460,0.076711,True
1,0.7,Zokekegua Long,6644582427941,0.647,0.361,0.526,0.532,4353938110450,0.5290,0.4450,0.651,true,Billmanserver,pop,6644582427941,0.741140,False
2,0.7,Bames Caslay,3983808991145,0.394,0.496,0.448,0.602,3820091516344,0.5250,0.5105,0.361,true,Billmanserver,pop,3983808991145,0.835208,False
3,0.7,Bames Mesganj,8182730130672,0.456,0.555,0.613,0.496,3820091516344,0.5545,0.5548,0.541,true,Billmanserver,pop,8182730130672,0.692977,False
4,0.7,Bames Fordciubeitsu,4068023816136,0.436,0.575,0.420,0.529,3820091516344,0.4745,0.5248,0.399,true,Billmanserver,pop,4068023816136,0.386875,True
5,0.7,Zokekegua Bang,6354374692878,0.467,0.288,0.439,0.532,4353938110450,0.4855,0.3868,0.571,true,Billmanserver,pop,6354374692878,0.061187,True
6,0.7,Zokekegua Piakoy,0657651821206,0.570,0.461,0.428,0.548,4353938110450,0.4880,0.4745,0.575,true,Billmanserver,pop,0657651821206,0.946227,False
7,0.7,Rayahhun Frantiae,7603049807934,0.634,0.356,0.504,0.493,5821792704095,0.4985,0.4272,0.613,true,william.jeffrey.harding@gmail.com,pop,7603049807934,0.303091,True
8,0.7,Kalgagerkal Bedeldepark,0653635598459,0.202,0.462,0.479,0.600,6730711275117,0.5395,0.5008,0.056,true,william.jeffrey.harding@gmail.com,pop,0653635598459,0.469182,True
9,0.7,Rayahhun Dorf,3760594787864,0.538,0.310,0.549,0.360,5821792704095,0.4545,0.3822,0.746,true,william.jeffrey.harding@gmail.com,pop,3760594787864,0.506446,True


In [38]:
reproducing_pops

,health,name,objid,conformity,literacy,aggression,constitution,isIn,industry,wealth,factionLoyalty,isIdle,username,objtype,id
0,0.7,Bames Todin,4376681497460,0.408,0.469,0.474,0.380,3820091516344,0.4270,0.4480,0.373,true,Billmanserver,pop,4376681497460
4,0.7,Bames Fordciubeitsu,4068023816136,0.436,0.575,0.420,0.529,3820091516344,0.4745,0.5248,0.399,true,Billmanserver,pop,4068023816136
5,0.7,Zokekegua Bang,6354374692878,0.467,0.288,0.439,0.532,4353938110450,0.4855,0.3868,0.571,true,Billmanserver,pop,6354374692878
7,0.7,Rayahhun Frantiae,7603049807934,0.634,0.356,0.504,0.493,5821792704095,0.4985,0.4272,0.613,true,william.jeffrey.harding@gmail.com,pop,7603049807934
8,0.7,Kalgagerkal Bedeldepark,0653635598459,0.202,0.462,0.479,0.600,6730711275117,0.5395,0.5008,0.056,true,william.jeffrey.harding@gmail.com,pop,0653635598459
9,0.7,Rayahhun Dorf,3760594787864,0.538,0.310,0.549,0.360,5821792704095,0.4545,0.3822,0.746,true,william.jeffrey.harding@gmail.com,pop,3760594787864
10,0.7,Rayahhun Rahpia,2977486889939,0.594,0.504,0.392,0.455,5821792704095,0.4235,0.4638,0.380,true,william.jeffrey.harding@gmail.com,pop,2977486889939
12,0.7,Rayahhun Pon,5180713039130,0.430,0.403,0.655,0.453,5821792704095,0.5540,0.4785,0.674,true,william.jeffrey.harding@gmail.com,pop,5180713039130
15,0.7,Obattla Pe,5100638967625,0.325,0.352,0.606,0.450,8822366435138,0.5280,0.4400,0.459,true,BillmanLocal2,pop,5100638967625
16,0.7,Obattla Van,1800951322768,0.446,0.390,0.549,0.573,8822366435138,0.5610,0.4755,0.423,true,BillmanLocal2,pop,1800951322768


In [39]:
nodes = []
edges = []
# Event edges must be uploaded separately, because the event nodes haven't been created yet.    
event_edges = []

In [40]:

for i in reproducing_pops.index.to_list():
    p = reproducing_pops.loc[i].to_dict()
    f = factions_df.loc[i].to_dict()
    species = species_df.loc[i].to_dict()
    location = locations_df.loc[i].to_dict()
    child = growth.grow_pop(p,species,params,syllables)    
    pop_locations = growth.assign_pop_to_faction(f)    
    c.patch_property(f['objid'], 'pop_locations', str(pop_locations))
    nodes.append(child)
    event = growth.population_growth_event(p, location,child, params)
    nodes.append(event)
    edges.append({"node1": child["objid"], "node2": p["objid"], "label": "childOf"})
    edges.append({"node1": child["objid"], "node2": child["isIn"], "label": "isIn"})
    edges.append({"node1": child["objid"], "node2": species["objid"], "label": "isOf"})
    edges.append({"node1": child["objid"], "node2": location["objid"], "label": "inhabits"})
    event_edges.append(c.create_custom_edge(event,location,'happenedAt'))
    event_edges.append(c.create_custom_edge(p,event,'caused'))


In [41]:
c.patch_property(f['objid'], 'pop_locations', str(pop_locations))
c.res



In [42]:
p

{'health': 0.7,
 'name': 'Obattla Ser',
 'objid': '7828874183454',
 'conformity': 0.372,
 'literacy': 0.37,
 'aggression': 0.575,
 'constitution': 0.581,
 'isIn': '8822366435138',
 'industry': 0.578,
 'wealth': 0.474,
 'factionLoyalty': 0.424,
 'isIdle': 'true',
 'username': 'BillmanLocal2',
 'objtype': 'pop',
 'id': '7828874183454'}

In [43]:
f

{'name': 'Obattla',
 'objid': '8822366435138',
 'lat': -0.149,
 'long': 0.0,
 'pop_loactions': nan,
 'username': 'BillmanLocal2',
 'objtype': 'faction',
 'id': '8822366435138',
 'pop_locations': '[[0, 0], [-1, 1], [-2, 0], [0, 2], [-1, -1], [2, -2], [1, -1]]'}

In [44]:
growth.grow_pop(p,species,params,syllables)

{'health': 0.42,
 'name': 'Obattla Sernew',
 'objid': '0311923096522',
 'conformity': 0.466,
 'literacy': 0.31,
 'aggression': 0.669,
 'constitution': 0.545,
 'isIn': '8822366435138',
 'industry': 0.347,
 'wealth': 0.269,
 'factionLoyalty': 0.444,
 'isIdle': 'true',
 'username': 'BillmanLocal2',
 'objtype': 'pop',
 'id': '0311923096522',
 'label': 'pop'}

In [45]:
pd.DataFrame.from_records(nodes)


,health,name,objid,conformity,literacy,aggression,constitution,isIn,industry,wealth,factionLoyalty,isIdle,username,objtype,id,label,text,visibleTo,time
0,0.42,Bames Todindan,3758791779668,0.468,0.426,0.390,0.318,3820091516344,0.256,0.273,0.309,true,Billmanserver,pop,3758791779668,pop,NaN,NaN,NaN
1,NaN,population growth,6873248824450,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,event,NaN,NaN,event,The population (Bames Todin) inhabiting Townna...,Billmanserver,28290.0
2,0.42,Bames Fordciubeitsuat,8319579269796,0.421,0.493,0.419,0.435,3820091516344,0.285,0.292,0.452,true,Billmanserver,pop,8319579269796,pop,NaN,NaN,NaN
3,NaN,population growth,2167592496448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,event,NaN,NaN,event,The population (Bames Fordciubeitsu) inhabitin...,Billmanserver,28290.0
4,0.42,Zokekegua Bangos,5688339314898,0.392,0.251,0.349,0.559,4353938110450,0.291,0.254,0.621,true,Billmanserver,pop,5688339314898,pop,NaN,NaN,NaN
5,NaN,population growth,3067062517071,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,event,NaN,NaN,event,The population (Zokekegua Bang) inhabiting Tow...,Billmanserver,28290.0
6,0.42,Rayahhun Frantiaevel,2865443477930,0.579,0.290,0.590,0.535,5821792704095,0.299,0.201,0.642,true,william.jeffrey.harding@gmail.com,pop,2865443477930,pop,NaN,NaN,NaN
7,NaN,population growth,8842376357522,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,event,NaN,NaN,event,The population (Rayahhun Frantiae) inhabiting ...,william.jeffrey.harding@gmail.com,28290.0
8,0.42,Kalgagerkal Bedeldeparkdel,3566990791371,0.119,0.509,0.535,0.571,6730711275117,0.324,0.335,0.056,true,william.jeffrey.harding@gmail.com,pop,3566990791371,pop,NaN,NaN,NaN
9,NaN,population growth,5430424743023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,event,NaN,NaN,event,The population (Kalgagerkal Bedeldepark) inhab...,william.jeffrey.harding@gmail.com,28290.0


In [38]:
pd.DataFrame.from_records(edges)

,node1,node2,label
0,0287502876706,5225645311508,childOf
1,0287502876706,1066241530501,isIn
2,0287502876706,8132528530336,isOf
3,3894182758189,9525508045282,childOf
4,3894182758189,1066241530501,isIn
5,3894182758189,8132528530336,isOf
6,0887781405662,3539278056453,childOf
7,0887781405662,1066241530501,isIn
8,0887781405662,8132528530336,isOf
9,8477282992499,4915654495902,childOf


In [39]:
event_edges

["\n        g.V().has('objid','2464638614452')\n            .addE('happenedAt')\n            .to(g.V().has('objid','7946732117793'))\n        ",
 "\n        g.V().has('objid','5225645311508')\n            .addE('caused')\n            .to(g.V().has('objid','2464638614452'))\n        ",
 "\n        g.V().has('objid','4122996995167')\n            .addE('happenedAt')\n            .to(g.V().has('objid','7946732117793'))\n        ",
 "\n        g.V().has('objid','9525508045282')\n            .addE('caused')\n            .to(g.V().has('objid','4122996995167'))\n        ",
 "\n        g.V().has('objid','9315204094813')\n            .addE('happenedAt')\n            .to(g.V().has('objid','7946732117793'))\n        ",
 "\n        g.V().has('objid','3539278056453')\n            .addE('caused')\n            .to(g.V().has('objid','9315204094813'))\n        ",
 "\n        g.V().has('objid','5552970289799')\n            .addE('happenedAt')\n            .to(g.V().has('objid','7946732117793'))\n        

In [34]:
growth.grow(c,params,syllables)